<h1 align='center'>Construindo bons conjuntos de dados de treinamento - pré-processamento de dados</h1>
<p align= 'center'><img src=https://sloboda-studio.com/wp-content/uploads/2020/08/Group-192.jpg></p>
<p align= 'justify'>A qualidade dos dados e a quantidade de informações úteis que eles contêm são fatores-chave que determinam o quão bem um algoritmo de aprendizado de máquina pode aprender. Portanto, é absolutamente crítico garantir que examinemos e pré-processemos um conjunto de dados antes de alimentá-lo a um algoritmo de aprendizado.</p>

<h3>Lidando com dados perdidos</h3>
<p align= 'justify'>Não é incomum em aplicações do mundo real que nossos exemplos de treinamento estejam faltando um ou mais valores por vários motivos. Pode ter havido um erro no processo de coleta de dados, certas medidas podem não ser aplicáveis ou campos específicos podem ter sido simplesmente deixados em branco em uma pesquisa, por exemplo. Normalmente, vemos valores ausentes como espaços em branco em nossa tabela de dados ou como strings de espaço reservado, como <i>NaN</i>, que significa "não é um número" ou <i>NULL</i> (um indicador comumente usado de valores desconhecidos em bancos de dados relacionais). Infelizmente, a maioria das ferramentas computacionais são incapazes de lidar com esses valores ausentes ou produzirão resultados imprevisíveis se simplesmente os ignorarmos. Portanto, é crucial que cuidemos desses valores ausentes antes de prosseguirmos com análises adicionais.</p>

<h3>Identificando valores ausentes em dados tabulares</h3>

In [2]:
import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


<p align='justify'>Para um DataFrame maior, pode ser tedioso procurar manualmente os valores ausentes; neste caso, podemos usar o método <i>isnull</i> para retornar um DataFrame com valores booleanos que indicam se uma célula contém um valor numérico <i>(False)</i> ou se faltam dados <i>(True)</i>. Usando o método <i>sum</i>, podemos retornar o número de valores ausentes por coluna da seguinte forma:</p>

In [3]:
# O somatório das colunas que possuem dados NaN ou Nulos.
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

<h4>Eliminando exemplos de treinamento ou recursos com valores ausentes</h4>

In [4]:
# Removendo todas as linhas que tem Nan
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [5]:
# Removendo todas as colunas que tem Nan
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [6]:
# Removendo as linhas se TODAS forem NaN.
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [7]:
# Removendo as linhas se tiver pelo menos que 4 valores reais.
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [9]:
# Removendo as linhas aonde aparece NaN em uma coluna específica, no caso "C".
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


<p align='justify'>Embora a remoção de dados ausentes pareça ser uma abordagem conveniente, ela também apresenta algumas desvantagens; por exemplo, podemos acabar removendo muitas amostras, o que impossibilitará uma análise confiável. Ou, se removermos muitas colunas de <i>features</i>, corremos o risco de perder informações valiosas que nosso classificador precisaria para discriminar entre as classes. Uma das alternativas mais usadas para lidar com valores ausentes é chamada de <b>Técnica de Interpolação</b>.</p>

<h3>Como imputar valores ausentes</h3>
<p align='justify'>Como foi dito anteriomente, excluir as linhas ou colunas pode comprometer dados valiosos, no conjunto como um todo. Como alteranativa, podemos usar diferentes técnicas de interpolação para estimar os valores ausentes dos outros exemplos de treinamento em nosso conjunto de dados. Uma das técnicas de interpolação mais comuns é a imputação média, onde simplesmente substituímos o valor ausente pelo valor médio de toda a coluna de <i>features</i>. Uma maneira conveniente de conseguir isso é usando a classe <i>SimpleImputer</i> do <i>scikit-learn</i>, conforme mostrado no código a seguir:</p>

In [10]:
# Conjunto de dados
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [11]:
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='mean') # Utilizando a MÉDIA. Poderíamos utilizar: "median"(Mediana) ou 'most_frequent'(Mais Frequente)

# O "most_frequent" é usando para colunas categóricas: cores, sexo, tamanho (P,M,G), entre outras.


imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [12]:
# Alternativamente poderíamos usar o Pandas "fillna" para prover o mesmo recurso.

df.fillna(df.mean()) 

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


<h3>Entendendo a API do estimador scikit-learn</h3>
<p align='justify'>Anteriormente, usamos a classe <i>SimpleImputer</i> do <i>scikit-learn</i> para imputar valores ausentes em nosso conjunto de dados. A classe <i>SimpleImputer</i> pertence às chamadas classes de transformadores no <i>scikit-learn</i>, que são usadas para transformação de dados. Os dois métodos essenciais desses estimadores são o ajuste e a transformação. O método <i>fit</i> é usado para aprender os parâmetros dos dados de treinamento e o método <i>transform</i> usa esses parâmetros para transformar os dados. Qualquer matriz de dados a ser transformada precisa ter o mesmo número de recursos que a matriz de dados que foi usada para ajustar o modelo.
<p align='center'><img src=https://www.bogotobogo.com/python/scikit-learn/images/scikit-data-processing1-missing-data/fit-transform-scikit-learn-estimator.png></p>


<p align='justify'>Os classificadores que usamos pertencem aos chamados estimadores do <i>scikit-learn</i>, com uma <i>API</i> conceitualmente muito semelhante à classe do transformador. Os estimadores têm um método de previsão, mas também podem ter um método de transformação. Como você deve se lembrar, também usamos o método de <i>fit</i> para aprender os parâmetros de um modelo quando treinamos esses estimadores para classificação. No entanto, em tarefas de aprendizado supervisionado, também fornecemos os rótulos de classe para ajustar o modelo, que podem ser usados para fazer previsões sobre novos exemplos de dados não rotulados por meio do método de previsão, conforme ilustrado na figura a seguir:</p>
<p align='center'><img src=https://www.bogotobogo.com/python/scikit-learn/images/scikit-data-processing1-missing-data/scikit-learn-predict-method.png></p>

<h4>Como lidar com dados categóricos</h4>
<p align='justify'>Até agora, trabalhamos apenas com valores numéricos. No entanto, não é incomum que conjuntos de dados do mundo real contenham uma ou mais colunas de recursos categóricos. Agora, faremos uso de exemplos simples, mas eficazes, para ver como lidar com esse tipo de dados em bibliotecas de computação numérica. Quando estamos falando de dados categóricos, temos que distinguir ainda mais entre características <b>ordinais e nominais</b>. Os recursos ordinais podem ser entendidos como valores categóricos que podem ser classificados ou ordenados. Por exemplo, o tamanho da camiseta seria um recurso ordinal, pois podemos definir uma ordem: XL > L > M. Em contrapartida, os recursos nominais não implicam em nenhuma ordem e, para continuar com o exemplo anterior, poderíamos pensar em a cor da camiseta como um recurso nominal, pois normalmente não faz sentido dizer que, por exemplo, o vermelho é maior que o azul.</p>

In [13]:
# Criando um Conjunto de dados com classes categóricas

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [14]:
# Mapeando recursos ordinais
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [15]:
# Se quiser reverter aos valores aprendidos.
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

<h4>Codificando rótulos de classe</h4>
<p align='justify'>Muitas bibliotecas de aprendizado de máquina exigem que os rótulos de classe sejam codificados como valores inteiros. Embora a maioria dos estimadores para classificação no <i>scikit-learn</i> converta rótulos de classe em inteiros internamente. Considera-se uma boa prática fornecer rótulos de classe como matrizes de inteiros para evitar falhas técnicas. Para codificar os rótulos de classe, podemos usar uma abordagem semelhante ao mapeamento de recursos ordinais discutidos anteriormente. Precisamos lembrar que os rótulos de classe não são ordinais e não importa qual número inteiro atribuímos a um rótulo de <i>string</i> específico. Assim, podemos simplesmente enumerar os rótulos das classes, começando em 0:</p>

In [16]:
import numpy as np

# Crie um dict de mapeamento para converter rótulos de classe de strings para inteiros

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [17]:
# Agora podemos converter os rótulos para converter rótulos de classe strings para inteiros
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [18]:
# Para reverter

inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [19]:
# Alternativamente, podemos usar o LabelEncoder
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])